In [1]:
# x_new = x - alpha * gradient(x)
import numpy as np

In [2]:
def gradient_descent(f, gradient, x0, alpha, eps, max_iter):
    x = x0
    
    for i in range(max_iter):
        x_new = x - alpha * gradient(x)
        
        if np.abs(f(x_new) - f(x)) < eps:
            break
            
        x = x_new
        
    converged = i != max_iter
    result = {}
    result['converged'] = converged
    result['num_iter'] = i
    result['x'] = x_new
    
    return result

In [3]:
def f(x):
    return 0.5 * (x[0] ** 2 + 10*x[1]**2)

In [4]:
def gradient(x):
    return np.array([x[0], 10 * x[1]])

In [11]:
x0 = np.array([3, 5])
alpha = 0.1
eps = 0.001
max_iter = 1000

gradient_descent(f, gradient, x0, alpha, eps, max_iter)

{'converged': True, 'num_iter': 33, 'x': array([0.08343852, 0.        ])}

In [7]:
def momentum(f, gradient, x0, alpha, eps, max_iter, beta):
    x = x0
    d = 0
    
    for i in range(max_iter):
        d = beta * d + alpha * gradient(x)
        x_new = x - d
        
        if np.abs(f(x_new) - f(x)) < eps:
            break
            
        x = x_new
        
    converged = i != max_iter
    result = {}
    result['converged'] = converged
    result['num_iter'] = i
    result['x'] = x_new
    
    return result

In [9]:
momentum(f, gradient, x0, alpha, eps, max_iter, beta=0.5)

{'converged': True, 'num_iter': 16, 'x': array([ 0.00486534, -0.00709534])}

In [12]:
def nesterov(f, gradient, x0, alpha, eps, max_iter, beta):
    x = x0
    d = 0
    
    for i in range(max_iter):
        d = beta * d + alpha * gradient(x - beta * d)
        x_new = x - d
        
        if np.abs(f(x_new) - f(x)) < eps:
            break
            
        x = x_new
        
    converged = i != max_iter
    result = {}
    result['converged'] = converged
    result['num_iter'] = i
    result['x'] = x_new
    
    return result

In [13]:
nesterov(f, gradient, x0, alpha, eps, max_iter, beta=0.9)

{'converged': True, 'num_iter': 25, 'x': array([-0.04738745,  0.        ])}

n-ti momenat: E(X^n)
E(X)
E(X^2)

$$ m_0 = 0 $$
$$ m_1 = \beta_1 m_0 + (1 - \beta_1)g_1 = (1 - \beta_1)g_1 $$
$$ m_2 = \beta_1 m_1 + (1 - \beta_1)g_2 = \beta_1 (1 - \beta_1)g_1  + (1 - \beta_1)g_2 $$
$$ m_3 = \beta_1 m_2 + (1 - \beta_1)g_3 =  \beta_1 ^ 2(1 - \beta_1)g_1  + \beta_1(1 - \beta_1)g_2 + (1 - \beta_1)g_3 $$

$$ m_t = (1 - \beta_1) \sum_{i=0}^t \beta_1^{t - i}g_i $$

$$ E[m_t] = E[(1 - \beta_1) \sum_{i=0}^t \beta_1^{t - i}g_i]
= E[g_t] [(1 - \beta_1) \sum_{i=0}^t \beta_1^{t - i}  + greska$$

$$ (1 - \beta_1) \sum_{i=0}^t \beta_1^{t - i} = 1 - \beta_1 ^t $$

In [15]:
def adam(f, gradient, x0, alpha, eps, max_iter, beta1, beta2, delta):
    m = 0
    v = 0
    x = x0
    
    for i in range(1, max_iter):
        g = gradient(x)
        m = beta1 * m + (1 - beta1) * g
        v = beta2 * v + (1 - beta2) * g ** 2
        
        m_hat = m / (1 - beta1 ** i)
        v_hat = v / (1 - beta2 ** i)
        
        x_new = x - alpha * (m_hat / (np.sqrt(v_hat) + delta))
        
        if np.abs(f(x_new) - f(x)) < eps:
            break
            
        x = x_new
        
    converged = i != max_iter
    result = {}
    result['converged'] = converged
    result['num_iter'] = i
    result['x'] = x_new
    
    return result

In [16]:
adam(f, gradient, x0, alpha, eps, max_iter, beta1=0.9, beta2=0.999, delta=1e-7)

{'converged': True, 'num_iter': 86, 'x': array([0.01802498, 0.00438169])}

In [17]:
# Branch and Bound

In [18]:
# Q---
# ---
# -Q--
# ----

In [19]:
def print_solution(board):
    n = board.shape[0]
    for i in range(n):
        for j in range(n):
            print(board[i][j], end=' ')
        print()

In [20]:
def solve(n):
    board = np.full((n, n), '.', dtype=str)
    
    row_check = np.full(n, False)
    
    d1_check = np.full(2 * n - 1, False)
    
    d2_check = np.full(2 * n - 1, False)
    
    if not bnb(board, 0, row_check, d1_check, d2_check):
        print('Nema resenja')
        return False
    print_solution(board)

In [21]:
def bnb(board, c, row_check, d1_check, d2_check):
    n = board.shape[0]
    
    if c >= n:
        return True
    
    for r in range(n):
        if is_free(r, c, row_check, d1_check, d2_check):
            board[r][c] = 'Q'
            row_check[r] = True
            d1_check[r + c] = True
            d2_check[r - c + n - 1] = True
            
            if bnb(board, c + 1, row_check, d1_check, d2_check):
                return True
            
            board[r][c] = '.'
            row_check[r] = False
            d1_check[r + c] = False
            d2_check[r - c + n - 1] = False
            
    return False

In [22]:
def is_free(r, c, row_check, d1_check, d2_check):
    n = row_check.shape[0]
    if row_check[r] or d1_check[r + c] or d2_check[r - c + n - 1]:
        return False
    return True

In [23]:
solve(8)

Q . . . . . . . 
. . . . . . Q . 
. . . . Q . . . 
. . . . . . . Q 
. Q . . . . . . 
. . . Q . . . . 
. . . . . Q . . 
. . Q . . . . . 
